## Basic Imports

In [4]:
import matplotlib
import pandas as pd

## Analysis

In [5]:
OG_DIHARD_AND_PYANNOTE = "../data/pyannote_original_vad_results.csv"
ENHANCED_DIHARD_AND_PYANNOTE = "../data/pyannote_MPSENet_enhanced_vad_results.csv"

OG_DIHARD_AND_SILERO = "../data/silero_original_vad_results.csv"
ENHANCED_DIHARD_AND_SILERO = "../data/silero_MPSENet_enhanced_vad_results.csv"

In [23]:
df_pyannote_original = pd.read_csv(OG_DIHARD_AND_PYANNOTE)
df_pyannote_enhanced = pd.read_csv(ENHANCED_DIHARD_AND_PYANNOTE)

df_silero_original = pd.read_csv(OG_DIHARD_AND_SILERO)
df_silero_enhanced = pd.read_csv(ENHANCED_DIHARD_AND_SILERO)

### Overall results

In [24]:
import pandas as pd

# Combine all results
df_pyannote_original['model'] = 'pyannote'
df_pyannote_enhanced['model'] = 'pyannote'
df_silero_original['model'] = 'silero'
df_silero_enhanced['model'] = 'silero'

df_pyannote_original['type'] = 'original'
df_pyannote_enhanced['type'] = 'enhanced'
df_silero_original['type'] = 'original'
df_silero_enhanced['type'] = 'enhanced'

df_all = pd.concat([df_pyannote_original, df_pyannote_enhanced, df_silero_original, df_silero_enhanced])

# Compute overall metrics
overall_metrics = df_all.groupby(['model', 'type']).agg({
    'detection_accuracy': ['mean', 'std'],
    'detection_error_rate': ['mean', 'std'],
    'missed_detection_rate': ['mean', 'std'],
    'false_alarm_rate': ['mean', 'std'],
    'roc_auc': ['mean', 'std']
}).reset_index()

# Rename columns for readability
overall_metrics.columns = ['_'.join(col).strip() if col[1] else col[0] for col in overall_metrics.columns]

# Save to CSV
overall_metrics.to_csv("../data/overall_vad_metrics.csv", index=False)

# Display results
overall_metrics


,model,type,detection_accuracy_mean,detection_accuracy_std,detection_error_rate_mean,detection_error_rate_std,missed_detection_rate_mean,missed_detection_rate_std,false_alarm_rate_mean,false_alarm_rate_std,roc_auc_mean,roc_auc_std
0,pyannote,enhanced,0.821005,0.170000,0.244601,0.236586,19.643441,22.072368,4.816643,9.975015,0.835394,0.105345
1,pyannote,original,0.927434,0.054386,0.104874,0.126775,3.707213,3.537448,6.780213,11.706428,0.875761,0.085593
2,silero,enhanced,0.763657,0.202685,0.324358,0.279347,26.188732,29.359101,6.247027,10.188715,0.757592,0.098799
3,silero,original,0.871213,0.106919,0.181780,0.173639,10.759598,15.487625,7.418384,10.549849,0.814694,0.076064


### Top 5 differing samples

In [25]:
# Merge original and enhanced results on flac_file and vad_model
df_pyannote_enhanced['flac_file'] = df_pyannote_enhanced['flac_file'].str.replace('.wav', '.flac', regex=False)
df_silero_enhanced['flac_file'] = df_silero_enhanced['flac_file'].str.replace('.wav', '.flac', regex=False)

df_pyannote = df_pyannote_original.merge(df_pyannote_enhanced, on=['flac_file', 'vad_model'], suffixes=('_orig', '_enh'))
df_silero = df_silero_original.merge(df_silero_enhanced, on=['flac_file', 'vad_model'], suffixes=('_orig', '_enh'))

# Function to compute differences and sort
def compute_differences(df, vad_model):
    df['accuracy_diff'] = abs(df['detection_accuracy_orig'] - df['detection_accuracy_enh'])
    df['der_diff'] = abs(df['detection_error_rate_orig'] - df['detection_error_rate_enh'])
    df['mdr_diff'] = abs(df['missed_detection_rate_orig'] - df['missed_detection_rate_enh'])
    df['far_diff'] = abs(df['false_alarm_rate_orig'] - df['false_alarm_rate_enh'])

    # Sort by differences in the specified order
    df_sorted = df.sort_values(by=['accuracy_diff', 'der_diff', 'mdr_diff', 'far_diff'], ascending=False)

    # Select top 5 differing samples
    top_5 = df_sorted.head(5)

    # Format the output
    formatted_output = top_5[['flac_file', 
                               'detection_accuracy_orig', 'detection_accuracy_enh', 
                               'detection_error_rate_orig', 'detection_error_rate_enh', 
                               'missed_detection_rate_orig', 'missed_detection_rate_enh', 
                               'false_alarm_rate_orig', 'false_alarm_rate_enh']]

    return formatted_output

# Compute differences for both pyannote and silero models
top_5_pyannote = compute_differences(df_pyannote, "pyannote")
top_5_pyannote

,flac_file,detection_accuracy_orig,detection_accuracy_enh,detection_error_rate_orig,detection_error_rate_enh,missed_detection_rate_orig,missed_detection_rate_enh,false_alarm_rate_orig,false_alarm_rate_enh
65,DH_EVAL_0066.flac,0.915533,0.212492,0.086731,0.808609,8.581084,80.860933,0.091988,0.000000
43,DH_EVAL_0044.flac,0.919253,0.238642,0.090244,0.850869,4.418995,84.688818,4.605453,0.398088
213,DH_EVAL_0214.flac,0.944102,0.339252,0.062032,0.733260,3.007091,72.649594,3.196100,0.676400
138,DH_EVAL_0139.flac,0.916921,0.320222,0.092548,0.757025,4.792917,75.287603,4.461871,0.414913
180,DH_EVAL_0181.flac,0.893566,0.324845,0.139810,0.886867,4.514353,88.227792,9.466626,0.458951


In [26]:
top_5_silero = compute_differences(df_silero, "silero")
top_5_silero

,flac_file,detection_accuracy_orig,detection_accuracy_enh,detection_error_rate_orig,detection_error_rate_enh,missed_detection_rate_orig,missed_detection_rate_enh,false_alarm_rate_orig,false_alarm_rate_enh
65,DH_EVAL_0066.flac,0.905060,0.043049,0.097492,0.982592,9.673919,98.259176,0.075301,0.000000
138,DH_EVAL_0139.flac,0.910394,0.160204,0.099824,0.935226,5.724160,93.466353,4.258273,0.056273
213,DH_EVAL_0214.flac,0.899947,0.158898,0.111033,0.933406,7.268457,93.209834,3.834863,0.130790
43,DH_EVAL_0044.flac,0.808851,0.120984,0.213622,0.982361,19.281898,98.209244,2.080308,0.026814
18,DH_EVAL_0019.flac,0.960333,0.304771,0.041048,0.719057,2.353999,71.688183,1.750844,0.217553
